In [0]:
#Creating delta tables for analysis

In [0]:
%sql
CREATE OR REPLACE TABLE delta.`/FileStore/Insurance_Proj/Gold_Layer/company_summary` AS
SELECT
  c.Company AS company_name
 ,count(ft.FileNo) AS total_claims
 ,cast(avg(ft.RecoveryAmount) AS DECIMAL(19,4))  AS avg_recovery_amount
 ,cast(sum(ft.RecoveryAmount) AS DECIMAL(19,4))  AS total_recovery_amount
 ,sum(CASE WHEN st.status='Closed' THEN 1 ELSE 0 END) AS closed_claims
 ,sum(CASE WHEN st.status!='Closed' THEN 1 ELSE 0 END) AS opened_claims
FROM delta.`/FileStore/Insurance_Proj/Silver_Layer/Delta_Tables/fact` ft 
join delta.`/FileStore/Insurance_Proj/Silver_Layer/Delta_Tables/company` AS c  on c.companyid=ft.companyid
join delta.`/FileStore/Insurance_Proj/Silver_Layer/Delta_Tables/status`  AS st on ft.statusid=st.statusid
group by Company


num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE delta.`/FileStore/Insurance_Proj/Gold_Layer/coverage_summary` AS
SELECT
    Coverage,
    SubCoverage,
    COUNT(DISTINCT FileNo) AS total_claims,
    CAST(SUM(ft.RecoveryAmount) AS DECIMAL(19,4)) AS total_recovery,
    CAST(AVG(ft.RecoveryAmount) AS DECIMAL(19,4)) AS avg_recovery,
    COUNT(CASE WHEN Status = 'Closed' THEN 1 END) AS closed_claims
FROM delta.`/FileStore/Insurance_Proj/Silver_Layer/Delta_Tables/fact` AS ft 
join delta.`/FileStore/Insurance_Proj/Silver_Layer/Delta_Tables/coverage/` AS cv on cv.coverageid=ft.coverageid
join delta.`/FileStore/Insurance_Proj/Silver_Layer/Delta_Tables/subcoverage/` AS sc on sc.subcoverageid=ft.subcoverageid
join delta.`/FileStore/Insurance_Proj/Silver_Layer/Delta_Tables/status`  AS st on ft.statusid=st.statusid
GROUP BY Coverage, SubCoverage;


num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE delta.`/FileStore/Insurance_Proj/Gold_Layer/reason_summary` AS
SELECT
    Reason AS reason,
    SubReason AS sub_reason,
    COUNT(DISTINCT `FileNo`) AS total_claims,
    CAST(SUM(ft.RecoveryAmount) AS DECIMAL(19,4)) AS total_recovery,
    COUNT(CASE WHEN Conclusion = 'Claim Settled' THEN 1 END) AS settled_claims,
    COUNT(CASE WHEN Conclusion = 'Claim Paid' THEN 1 END) AS paid_claims,
    COUNT(CASE WHEN Conclusion = 'Company Position Upheld' THEN 1 END) AS company_upheld_claims
FROM delta.`/FileStore/Insurance_Proj/Silver_Layer/Delta_Tables/fact` AS ft 
join delta.`/FileStore/Insurance_Proj/Silver_Layer/Delta_Tables/reason/` AS r on r.reasonid=ft.reasonid
join delta.`/FileStore/Insurance_Proj/Silver_Layer/Delta_Tables/subreason/` AS sr on sr.subreasonid=ft.subreasonid
join delta.`/FileStore/Insurance_Proj/Silver_Layer/Delta_Tables/conclusion/` AS cl on cl.conclusionid=ft.conclusionid
GROUP BY Reason, SubReason;

num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE delta.`/FileStore/Insurance_Proj/Gold_Layer/reason_summary/disposition_conclusion_summary` AS
SELECT
    Disposition,
    Conclusion,
    COUNT(DISTINCT `FileNo`) AS total_claims,
    CAST(SUM(ft.RecoveryAmount) AS DECIMAL(19,4)) AS total_recovery,
    COUNT(CASE WHEN Conclusion = 'Claim Settled' THEN 1 END) AS settled_claims,
    COUNT(CASE WHEN Conclusion = 'Company Position Upheld' THEN 1 END) AS upheld_claims
FROM delta.`/FileStore/Insurance_Proj/Silver_Layer/Delta_Tables/fact` AS ft 
join delta.`/FileStore/Insurance_Proj/Silver_Layer/Delta_Tables/disposition/` AS dp on dp.dispositionid=ft.dispositionid
join delta.`/FileStore/Insurance_Proj/Silver_Layer/Delta_Tables/conclusion/` AS cl on cl.conclusionid=ft.conclusionid
GROUP BY Disposition, Conclusion;


num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE delta.`/FileStore/Insurance_Proj/Gold_Layer/time_based_claims_summary` AS
SELECT
    YEAR(ft.OpenedDate) AS opened_year,
    MONTH(ft.OpenedDate) AS opened_month,
    COUNT(DISTINCT FileNo) AS claims_opened,
    COUNT(CASE WHEN Status = 'Closed' THEN 1 END) AS claims_closed,
    CAST(SUM(ft.RecoveryAmount) AS DECIMAL(19,4)) AS total_recovery
FROM delta.`/FileStore/Insurance_Proj/Silver_Layer/Delta_Tables/fact` AS ft 
join delta.`/FileStore/Insurance_Proj/Silver_Layer/Delta_Tables/status`  AS st on ft.statusid=st.statusid
GROUP BY YEAR(ft.OpenedDate), MONTH(ft.OpenedDate);


num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE delta.`/FileStore/Insurance_Proj/Gold_Layer/company_based_claims_summary` AS
SELECT
    c.Company as company,
    YEAR(ft.OpenedDate) AS opened_year,
    MONTH(ft.OpenedDate) AS opened_month,
    COUNT(DISTINCT FileNo) AS claims_opened,
    COUNT(CASE WHEN Status = 'Closed' THEN 1 END) AS claims_closed,
    CAST(SUM(ft.RecoveryAmount) AS DECIMAL(19,4)) AS total_recovery
FROM delta.`/FileStore/Insurance_Proj/Silver_Layer/Delta_Tables/fact` AS ft 
join delta.`/FileStore/Insurance_Proj/Silver_Layer/Delta_Tables/company` AS c  on c.companyid=ft.companyid
join delta.`/FileStore/Insurance_Proj/Silver_Layer/Delta_Tables/status`  AS st on ft.statusid=st.statusid
GROUP BY YEAR(ft.OpenedDate), MONTH(ft.OpenedDate), c.Company
order by c.Company

num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE delta.`/FileStore/Insurance_Proj/Gold_Layer/status_summary` AS
SELECT
    Status,
    COUNT(DISTINCT FileNo) AS total_claims,
    CAST(SUM(ft.RecoveryAmount) AS DECIMAL(19,4)) AS total_recovery
FROM delta.`/FileStore/Insurance_Proj/Silver_Layer/Delta_Tables/fact` AS ft 
join delta.`/FileStore/Insurance_Proj/Silver_Layer/Delta_Tables/status`  AS st on ft.statusid=st.statusid
GROUP BY Status;


num_affected_rows,num_inserted_rows
